In [2]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [3]:
# load data
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
data = fetch_olivetti_faces()
X = data.images
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

In [5]:
X_train.shape

(280, 4096)

In [6]:
X_test.shape

(120, 4096)

In [7]:
# Deep RandomForest
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/olivetti.json")
gc = GCForest(config)

In [9]:
# count class
len(np.unique(y_train))

40

In [10]:
      # X_enc is the concatenated predict_proba result of RandomForest

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
     

[ 2018-03-27 14:59:21,314][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4096)],y_train.shape=(280,),X_groups_test.shape=[(120, 4096)],y_test.shape=(120,)
[ 2018-03-27 14:59:21,317][cascade_classifier.fit_transform] group_dims=[4096]
[ 2018-03-27 14:59:21,318][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-27 14:59:21,319][cascade_classifier.fit_transform] group_ends=[4096]
[ 2018-03-27 14:59:21,321][cascade_classifier.fit_transform] X_train.shape=(280, 4096),X_test.shape=(120, 4096)
[ 2018-03-27 14:59:21,325][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4096), X_cur_test.shape=(120, 4096)
/home/kader/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[ 2018-03-27 14:59:22,604][kfold_wrap

In [11]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100))

[ 2018-03-27 15:01:33,384][cascade_classifier.transform] X_groups_test.shape=[(120, 4096)]
[ 2018-03-27 15:01:33,387][cascade_classifier.transform] group_dims=[4096]
[ 2018-03-27 15:01:33,388][cascade_classifier.transform] X_test.shape=(120, 4096)
[ 2018-03-27 15:01:33,391][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(120, 4096)


Test Accuracy of RandomForest = 90.000000 %


In [16]:
# predict_proba result of RandomForest + data
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-03-27 15:01:36,930][cascade_classifier.transform] X_groups_test.shape=[(120, 4096)]
[ 2018-03-27 15:01:36,933][cascade_classifier.transform] group_dims=[4096]
[ 2018-03-27 15:01:36,935][cascade_classifier.transform] X_test.shape=(120, 4096)
[ 2018-03-27 15:01:36,937][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(120, 4096)


In [17]:
X_test_enc.shape

(120, 4136)

In [18]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [19]:
X_train_new.shape

(280, 4136)

In [20]:
# Cascade RandomForest
config = load_json("./examples/olivettigc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-03-27 15:01:38,840][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4136)],y_train.shape=(280,),X_groups_test.shape=[(120, 4136)],y_test.shape=(120,)
[ 2018-03-27 15:01:38,843][cascade_classifier.fit_transform] group_dims=[4136]
[ 2018-03-27 15:01:38,845][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-27 15:01:38,846][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-03-27 15:01:38,848][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-03-27 15:01:38,852][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)
[ 2018-03-27 15:01:39,974][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-03-27 15:01:41,344][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-03-27 15:01:42,672][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - esti

In [21]:
def update(X_train,X_test):
    # update Estimators
    for i in range(20):
        M = np.delete(X_train, (-1), axis=1)
        X_train= np.delete(M, (-1), axis=1)
        M = np.delete(X_test, (-1), axis=1)
        X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100),i,'fois')
    

In [22]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/olivettigc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-03-27 15:02:30,173][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4136)],y_train.shape=(280,),X_groups_test.shape=[(120, 4136)],y_test.shape=(120,)
[ 2018-03-27 15:02:30,175][cascade_classifier.fit_transform] group_dims=[4136]
[ 2018-03-27 15:02:30,177][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-27 15:02:30,178][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-03-27 15:02:30,182][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-03-27 15:02:30,185][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)
[ 2018-03-27 15:02:31,319][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-03-27 15:02:32,656][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-03-27 15:02:33,981][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - esti

[ 2018-03-27 15:03:20,843][cascade_classifier.transform] group_dims=[4136]
[ 2018-03-27 15:03:20,845][cascade_classifier.transform] X_test.shape=(120, 4136)
[ 2018-03-27 15:03:20,847][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(120, 4136)
[ 2018-03-27 15:03:22,128][cascade_classifier.fit_transform] X_groups_train.shape=[(280, 4136)],y_train.shape=(280,),X_groups_test.shape=[(120, 4136)],y_test.shape=(120,)
[ 2018-03-27 15:03:22,133][cascade_classifier.fit_transform] group_dims=[4136]
[ 2018-03-27 15:03:22,135][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-27 15:03:22,136][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-03-27 15:03:22,137][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-03-27 15:03:22,143][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)


('Test Accuracy of RandomForest = 89.166667 %', 0, 'fois')


[ 2018-03-27 15:03:23,292][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-03-27 15:03:24,626][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-03-27 15:03:25,961][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.00%
[ 2018-03-27 15:03:27,188][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-03-27 15:03:28,487][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-03-27 15:03:29,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=92.31%
[ 2018-03-27 15:03:30,916][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=87.50%
[ 2018-03-27 15:03:32,144][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=87.5

[ 2018-03-27 15:04:18,498][cascade_classifier.fit_transform] group_ends=[4136]
[ 2018-03-27 15:04:18,499][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-03-27 15:04:18,506][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)


...

[ 2018-03-27 15:19:29,299][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-03-27 15:19:30,661][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-03-27 15:19:31,983][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.00%
[ 2018-03-27 15:19:33,185][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-03-27 15:19:34,456][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-03-27 15:19:35,653][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=92.31%
[ 2018-03-27 15:19:36,879][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=90.00%
[ 2018-03-27 15:19:38,059][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=82.5

[ 2018-03-27 15:20:20,335][cascade_classifier.fit_transform] X_train.shape=(280, 4136),X_test.shape=(120, 4136)
[ 2018-03-27 15:20:20,338][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(280, 4136), X_cur_test.shape=(120, 4136)


('Test Accuracy of RandomForest = 91.666667 %', 18, 'fois')


[ 2018-03-27 15:20:21,513][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=92.50%
[ 2018-03-27 15:20:22,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-03-27 15:20:24,151][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=92.00%
[ 2018-03-27 15:20:25,372][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-03-27 15:20:26,760][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=87.50%
[ 2018-03-27 15:20:28,019][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=94.87%
[ 2018-03-27 15:20:29,225][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=87.50%
[ 2018-03-27 15:20:30,434][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=87.50

('Test Accuracy of RandomForest = 90.833333 %', 19, 'fois')


# ('Test Accuracy of RandomForest = 91.666667 %', 18, 'fois')